In [3]:
import numpy as np
from sympy import ntt, intt, convolution
from sympy.ntheory.residue_ntheory import nthroot_mod

def poly_mult_nega_schoolbook(a, b, q):
    assert(len(a) == len(b))
    n = len(a)
    c = [0] * n
    for i in range(n):
        c[i] = sum([a[j] * b[i - j] for j in range(i + 1)]) \
             - sum([a[j] * b[n + i - j] for j in range(i + 1, n)])
    res = [x % q for x in c]
    return np.array(res, dtype=np.object_)

def ntt_(x, q):
    N = len(x)
    GEN_N_ROOTS = nthroot_mod(1, 2 * N, q, True) # 2d-th root
    FIRST_NON_TRIVIAL_8TH_ROOT = GEN_N_ROOTS[1]
    GEN_8 = FIRST_NON_TRIVIAL_8TH_ROOT
    GEN_8_POW = [pow(GEN_8, i, q) for i in range(N)]
    xx = [(x[i] * GEN_8_POW[i]) % q for i in range(len(x))]

    x_ntt = ntt(xx, q)
    return np.array(x_ntt)

def intt_(x, q):
    N = len(x)
    GEN_N_ROOTS = nthroot_mod(1, 2 * N, q, True) # 2d-th root
    FIRST_NON_TRIVIAL_8TH_ROOT = GEN_N_ROOTS[1]
    GEN_8 = FIRST_NON_TRIVIAL_8TH_ROOT
    GEN_8_INV = pow(GEN_8, -1, q)
    GEN_8_INV_POW = [pow(GEN_8_INV, i, q) for i in range(N)]
    # Convert back to coefficient form.
    xx = intt(x, q)
    return np.array([(xx[i] * GEN_8_INV_POW[i]) % q for i in range(len(xx))])

def poly_mult_cyclic(x, y, q):
    return intt(np.array(ntt(x, q)) * np.array(ntt(y, q)) % q, q)

def ntt_conv(x, q):
    N = len(x)
    xx = np.empty(shape=(2 * N,), dtype=np.int64)
    np.concatenate((x, np.zeros(N, dtype=np.int64)), out=xx)
    return np.array(ntt(xx, q))

def intt_conv(x, q):
    return np.array(intt(x, q))

def conv_to_nega(x, q):
    return (x[:len(x) // 2] - x[len(x) // 2:]) % q

q = 17
x = np.array([8, 7, 10, 5])
y = np.array([2, 6, 9, 13])
# should be [9, 6, 1, 16]
N = 4
x_neg = ntt_(x, q)
y_neg = ntt_(y, q)
xy_neg = (x_neg * y_neg) % q
xy_neg = intt_(xy_neg, q)

# check
xy_neg1 = poly_mult_nega_schoolbook(x, y, q)
assert(np.all(xy_neg == xy_neg1))

print(f'x = {x}')
print(f'y = {y}')
print(f'y_neg = {y_neg}')
print(f'x_neg = {x_neg}')
print(f'xy_neg = {xy_neg}')

# gives no leading zero: [16, 11, 15, 16,  7,  5, 14]
xy_conv_np = np.convolve(x, y) % q
# gives [16, 11, 15, 16,  7,  5, 14, 0]
xx = np.concatenate((x, [0] * len(x)))
yy = np.concatenate((y, [0] * len(y)))
x_conv = ntt_conv(x, q)
y_conv = ntt_conv(y, q)
xy_conv = x_conv * y_conv % q
xy_conv = intt_conv(xy_conv, q)

# check
xy_conv1 = convolution(xx, yy, prime=q)
assert(np.all(xy_conv == xy_conv1))

print(f'y_conv = {y_conv}')
print(f'x_conv = {x_conv}')
print(f'xy_conv = {xy_conv}')

print(f'xy_converted = {conv_to_nega(xy_conv, q)}')

x = [ 8  7 10  5]
y = [ 2  6  9 13]
y_neg = [ 1 11  7  6]
x_neg = [ 0  4 11  0]
xy_neg = [ 9  6  1 16]
y_conv = [13 11  4  7  9  6 16  1]
x_conv = [13  4  7 11  6  0  6  0]
xy_conv = [16 11 15 16  7  5 14  0]
xy_converted = [ 9  6  1 16]
